In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os

In [2]:
train_labels = pd.read_csv("train.csv")

### Visualize how the train labels look

In [3]:
train_labels[train_labels.eq(1).any(axis=1)].head(2)

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,0,0,0,0,0,0,1,1
8,1.2.826.0.1.3680043.8.498.10022796280698534221...,78,Male,CTA,0,0,0,0,0,1,0,0,0,0,0,0,0,1


### Visualize how the train localizers look

In [4]:
train_localizers = pd.read_csv("train_localizers.csv")
train_localizers.head(2)

,SeriesInstanceUID,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,"{'x': 194.87253141831238, 'y': 178.32675044883...",Right Middle Cerebral Artery


In [5]:
train_labels.shape ,train_localizers.shape

((4405, 18), (2286, 4))

Visualizing a square around the center of the aneurysm for all of the images in the folder
1. Get the SOPInstanceUID that matches the one in the image
2. Get the coordinates from that observation
3. Graph the image with a red square around the center of the aneurysm

In [6]:
# Get the SOPInstanceUID that matches the image
def check_file_names_against_dataframe(folder_path, dataframe, column_name):
    try:
        file_names = {os.path.splitext(f)[0] for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path,f))}
    except FileNotFoundError:
        print(f"Error: The folder '{folder_path}' was not found.")
        return
    df_values = set(dataframe[column_name].astype(str).tolist())
    matching_files = file_names.intersection(df_values)
    files_without_match = file_names.difference(df_values)
    print(file_names)

In [10]:
# Get the SOPInstanceUID that matches the image
def check_file_names_against_dataframe(folder_path, dataframe, column_name):
    all_file_names = set()
    try:
        for dirpath, dirnames, filenames in os.walk(folder_path):
            for filename in filenames:
                file_name_without_ext = os.path.splitext(filename)[0]
                all_file_names.add(file_name_without_ext)
                
    except FileNotFoundError:
        print(f"Error: The folder '{folder_path}' was not found.")
        return set()
    df_values = set(dataframe[column_name].astype(str).tolist())
    matching_files = file_names.intersection(df_values)
    files_without_match = file_names.difference(df_values)
    return matching_files

In [ ]:
check_file_names_against_dataframe('series', train_localizers, 'SOPInstanceUID')

In [7]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np

def visualize_dcm(file_path):
    """
    Visualizes a DICOM image using pydicom and matplotlib.

    Args:
        file_path (str): The path to the .dcm file.
    """
    try:
        # Read the DICOM file
        dicom_data = pydicom.dcmread(file_path)

        # Get the pixel data
        # The pixel_array attribute returns the pixel data as a NumPy array.
        if 'PixelData' in dicom_data:
            pixel_array = dicom_data.pixel_array

            # Check for multiple frames (e.g., in a CT or MRI series)
            if pixel_array.ndim > 2:
                # If there are multiple frames, we'll just display the first one.
                # You could loop through them to create an animation or a series of plots.
                image_data = pixel_array[0, :, :]
            else:
                image_data = pixel_array

            # Normalize the data if necessary (e.g., for 16-bit images)
            # This is often important for good contrast.
            if image_data.dtype != np.uint8:
                image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
                image_data = (255 * image_data).astype(np.uint8)

            # Display the image
            plt.figure(figsize=(8, 8))
            plt.imshow(image_data, cmap=plt.cm.gray) # Use a grayscale colormap
            plt.title(f"DICOM Image: {dicom_data.SeriesDescription if 'SeriesDescription' in dicom_data else 'No Series Description'}")
            plt.xlabel("X-axis")
            plt.ylabel("Y-axis")
            plt.show()

        else:
            print("Error: The DICOM file does not contain pixel data.")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [8]:
visualize_dcm('1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm')

Error: File not found at 1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm


In [9]:
import pydicom
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the DICOM file
ds = pydicom.dcmread('1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm')
pixel_array = ds.pixel_array

# Define the central coordinate and size of the square
center_x, center_y = 258.3621186176837, 261.359900373599  # Example coordinates
square_size = 50

# Calculate the top-left corner for the square
top_left_x = center_x - square_size // 2
top_left_y = center_y - square_size // 2

# Create a figure and axes
fig, ax = plt.subplots(1)

# Display the DICOM image
ax.imshow(pixel_array, cmap=plt.cm.bone)

# Create a rectangle patch
rect = patches.Rectangle(
    (top_left_x, top_left_y),
    square_size,
    square_size,
    linewidth=1,
    edgecolor='r',
    facecolor='none'
)

# Add the rectangle to the axes
ax.add_patch(rect)

# Set the title and show the plot
plt.title("DICOM Image with Square Highlight")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'

In [24]:
print("Hello")

Hello
